# Exploring Major Outages in the U.S

**Name(s)**: Junyi Xu and Sean Chan

**Website Link**: https://mofuwu.github.io/outages-analysis/

## Code

In [1]:
import pandas as pd
import numpy as np 
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [2]:
# load outage data
outage = pd.read_excel('outage.xlsx', skiprows=5, usecols=['U.S._STATE', 'POSTAL.CODE', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE'])
outage = outage.drop(index=0)
outage

FileNotFoundError: [Errno 2] No such file or directory: 'outage.xlsx'

### Assessment of Missingness

### NMAR

The column `CUSTOMERS.AFFECTED` may be NMAR. The electricity provider may choose not to report the number of customers affected if there are too few people influenced by the power outage. Additional data about the range of area under the influence of the power outage may be useful to explain the missingness of the number of customers affected. 

### Missingness dependencies

In [ ]:
def cal_missing_tvd(df, group, category, value):
    pivoted = df.pivot_table(index=group, columns=category, values=value, aggfunc='size')
    pivoted = pivoted.apply(lambda x: x/x.sum())
    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    return tvd

In [ ]:
# Perform missingness permutation test between percentage affected and cause as well as percentage affected and state
def missingness_permutation(df, value, group, *content):
    N = 1000
    shuffled = df.copy()
    shuffled['MISSINGNESS'] = df[value].isna()
    observed_tvd = cal_missing_tvd(shuffled, group, 'MISSINGNESS', value)
    tvds = np.empty(N, dtype=float)
    for i in range(N):
        shuffled[group] = np.random.permutation(shuffled[group])
        tvds[i] = cal_missing_tvd(shuffled, group, 'MISSINGNESS', value)
    fig = px.histogram(pd.DataFrame(tvds, columns=['TVD of ' + content[1]]), histnorm='probability density', 
                       title='Total Variance Distance of missing and not missing total '+ content[0],
                      )
    fig.add_vline(x=observed_tvd, line_color='red')
    p_value = (tvds >= observed_tvd).mean()
    return p_value, fig

In [ ]:
df = outage[['OUTAGE.DURATION', 'U.S._STATE']]
p_value, fig = missingness_permutation(df, 'OUTAGE.DURATION', 'U.S._STATE', 'outage duration conditioned on US states', 'outage duration')
print('p-value: ', p_value)
fig.show()
fig.write_html('plots/MAR_duration_state.html', include_plotlyjs='cdn')

The permutation test above checks the missingness dependency of power outage duration on the US states. A p-value of 0.252 indicats that the missingness of power outage duration is independent of the US states.

In [ ]:
df = outage[['OUTAGE.DURATION', 'CAUSE.CATEGORY']]
p_value, fig = missingness_permutation(df, 'OUTAGE.DURATION', 'CAUSE.CATEGORY', 'outage duration conditioned on categories of events', 'outage duration')
print('p-value: ', p_value)
fig.show()
fig.write_html('plots/MAR_duration_category.html', include_plotlyjs='cdn')

The permutation test above checks the missingness dependency of power outage duration on the category of events that cause the power outage. A p-value of 0.002 confirms this dependency. 

In [ ]:
df = outage[['DEMAND.LOSS.MW', 'CAUSE.CATEGORY']]
p_value, fig = missingness_permutation(df, 'DEMAND.LOSS.MW', 'CAUSE.CATEGORY', 'Peak demand loss conditioned on categories of events', 'Peak demand loss')
print('p-value: ', p_value)
fig.show()

The permutation test above checks the missingness dependency of demand loss on the category of the events that cause the power outage. A p-value of 0.0 confirms this dependency.

### Cleaning and EDA

In [ ]:
def prob_impute(df, cols):
    missingness = outage[cols[-1]].isna()
    fill_index = np.random.choice(outage[cols[-1]].dropna().index, missingness.sum())
    fill_values = outage.loc[fill_index, cols]
    for col in cols:
        df.loc[missingness, col] = fill_values[col].to_numpy()

In [ ]:
prob_impute(outage, ['RES.PERCEN', 'IND.PERCEN', 'COM.PERCEN'])
prob_impute(outage, ['RES.PRICE', 'IND.PRICE', 'COM.PRICE'])
outage['OUTAGE.DURATION'] = outage.groupby('CAUSE.CATEGORY')['OUTAGE.DURATION'].transform(lambda x: x.fillna(x.mean()))
outage['DEMAND.LOSS.MW'] = outage.groupby('CAUSE.CATEGORY')['DEMAND.LOSS.MW'].transform(lambda x: x.fillna(x.mean()))

outage.loc[:, ~outage.columns.isin(['U.S._STATE', 'POSTAL.CODE', 'CAUSE.CATEGORY'])] = outage.loc[:, ~outage.columns.isin(['U.S._STATE', 'POSTAL.CODE', 'CAUSE.CATEGORY'])].astype(float)

outage['RES.MONEY.LOST'] = round(outage['OUTAGE.DURATION']*outage['RES.PERCEN']*outage['DEMAND.LOSS.MW']*outage['RES.PRICE'] / 60000, 2)
outage['COM.MONEY.LOST'] = round(outage['OUTAGE.DURATION']*outage['COM.PERCEN']*outage['DEMAND.LOSS.MW']*outage['COM.PRICE'] / 60000, 2)
outage['IND.MONEY.LOST'] = round(outage['OUTAGE.DURATION']*outage['IND.PERCEN']*outage['DEMAND.LOSS.MW']*outage['IND.PRICE'] / 60000, 2)
outage['TOTAL.MONEY.LOST'] = outage['RES.MONEY.LOST'] + outage['COM.MONEY.LOST'] + outage['IND.MONEY.LOST']

print(outage.head())
print(outage.head().to_markdown())

Based on the result of the missingness dependencies, we perform within-group mean impuatation on the `DEMAND.LOSS.WM` and `OUTAGE.DURATION` for their dependency on `CAUSE.CATEGORY`. 

We also perferm probabilistic imputation on `RES.PERCEN`, `COM.PERCEN`, `IND.PERCEN`, `RES.PRICE`, `COM.PRICE`, `IND.PRICE`. As the percentages of the electricity consumption of three sectors should add up to 100%, we perform the probabilistic imputation of these three columns together to retain consistency.

For mathematical convenience, we change columns that have numerical values into type of float.

We adopt the money lost during each power outage event as the main measurement of financial severity, which gives very straightfoward assessment of the financial impact of the power outage on the electricity provider. The money lost is calculated by multiplying electricity demand loss (Megawatt), electricity price(cents/killowatt-hour), and duration of the outage events (minutes). We calculate the money lost from each sector as well as the total money lost, the resulting columns are `RES.MONEY.LOST`, `COM.MONEY.LOST`, `IND.MONEY.LOST`, `TOTAL.MONEY.LOST`. During the calculate, we take into acconut the inconsistency of units. The unit of the money lost is dollar.

In [ ]:
# Univariate Analysis
fig = px.histogram(outage, 
             x = 'RES.MONEY.LOST',
             histnorm='probability',
             title='Distribution of money lost in residential sector due to power outages',
             labels = {'RES.MONEY.LOST': 'Money lost in residential sector (Dollars)'},
            )

fig.show()
fig.write_html('plots/res_money_lost_hist.html', include_plotlyjs='cdn')

The histogram shows the distribution of the total money lost in residential sector during each power outage. More than half of the power outage events lead to money lost less than 10000 dollars. More than 25% of cases cause money lost between 10000 dollars and 30000 dollars. Only about 10% of the cases lead to more than 300000 dollars. The highest money lost in residential sector is about 2 million dollars.

In [ ]:
fig = px.histogram(outage, 
             x = 'COM.MONEY.LOST',
             histnorm='probability',
             title='Distribution of money lost in commercial sector due to power outages',
             labels = {'COM.MONEY.LOST': 'Money lost in commercial sector (Dollars)'},
            )

fig.show()
fig.write_html('plots/com_money_lost_hist.html', include_plotlyjs='cdn')

The histogram shows the distribution of the total money lost in commercial sector during each power outage. Similar to the distribution for residential sector, over 60% of the power outage events lead to money lost less than 10000 dollars. About 25% of cases, the money lost is between 10000 dollars and 30000 dollars. Only about 10% of the cases lead to more than 300000 dollars. The highest money lost in commercial sector is about 1.5 million dollars, slightly smaller than the highest money lost in residencial sector.

In [ ]:
fig = px.histogram(outage, 
             x = 'IND.MONEY.LOST',
             histnorm='probability',
             title='Distribution of money lost in industrial sector due to power outages',
             labels = {'IND.MONEY.LOST': 'Money lost in industrial sector (Dollars)'},
            )

fig.show()
fig.write_html('plots/ind_money_lost_hist.html', include_plotlyjs='cdn')

The histogram shows the distribution of the total money lost in industrial sector during each power outage. The shape of the distribution is similar to the other two sectors, but the money lost is in general significantly smaller. About 50% of the power outage events lead to money lost less than 2500 dollars. About 25% of cases, the money lost is between 2500 dollars and 7500 dollars. The rest 25% of the cases lead to more than 7500 dollars. The highest money lost in industrial sector is about 935000 dollars. 

In [ ]:
# Bivariate analysis
total_money_lost_by_cause = outage.groupby('CAUSE.CATEGORY').agg({'TOTAL.MONEY.LOST': 'mean'}).reset_index()
fig = px.bar(total_money_lost_by_cause, 
       y='TOTAL.MONEY.LOST', 
       x='CAUSE.CATEGORY', 
       title='Average money lost during power outages due to different categories of events',
       labels = {'TOTAL.MONEY.LOST': 'Average money lost (Billion Dollars)', 'CAUSE.CATEGORY': 'Categories of events'},
      )
fig.show()
fig.write_html('plots/total_money_lost_vs_cause_category.html', include_plotlyjs='cdn')

The barchart shows that there are remarkable differences between the average money lost under different categories of events that cause the power outage. Power outage that is induced by fuel supply emergency and by islanding leads to significantly higher money lost on average than other categories of events. Based on this plot, we can conclude that categories of events is a key characteristic that determines the financial severity of the power outage for the electricity provider. Among all categories, fuel supply emergency and islanding worth particular attention because of their tendency to cause huge amount of money lost.

In [ ]:
total_money_lost_by_state = outage.groupby(['U.S._STATE', 'POSTAL.CODE']).agg({'TOTAL.MONEY.LOST': 'mean'}).reset_index()

fig = px.choropleth(total_money_lost_by_state, 
                    locations='POSTAL.CODE',
                    color='TOTAL.MONEY.LOST',
                    hover_name="U.S._STATE",
                    color_continuous_scale=px.colors.sequential.Purples,
                    scope='usa',
                    locationmode='USA-states',
                    title='Average money lost in different state due to power outages from January 2000 to July 2016')
fig.show()
fig.write_html('plots/total_money_lost_geospatial.html', include_plotlyjs='cdn')

Besides the categories of events that cause the power outage, we also look into the geographical factors. The choropleth graph depicts the total money lost due to power outage in each US state. Over the different states, we can observe some variations in the average money lost. The total financial lost is significantly higher in New York and North Dakota. Texas and Florida also have relatively high money lost. Many areas on the west coast show greater money lost than east coast and the middle states. In general, there are some variations in the money lost across different states, so US states may be another factor that influence the severity of financial lost.

In [ ]:
# Aggregates
cause_sector_table = outage[['RES.MONEY.LOST', 'COM.MONEY.LOST', 'IND.MONEY.LOST', 'CAUSE.CATEGORY']].groupby('CAUSE.CATEGORY').mean()
print(cause_sector_table.to_markdown())

Given that the distributions of money lost in residential, commercial, and industrial sector are very similar, it is hard to tell whether the money lost in one sector is different from the other. In respond to this, we plot a pivot table to investigate the difference between money lost of three sectors under different categories of events that cause the power outage. This can also help us explore the pattern of financial lost due to different categories of events for each sector respectively.

In this pivot table, we observe that the average money lost from industrial sector is significantly smaller under all categories of events than from the other two sectors, which is consistent with the conclusion we draw in the univariate analysis. The average money lost in residential and commercial sectors are very close. There is a slightly higher average money lost in residential sector due to equipment failure, intentional attack, public appeal, severe whether, and system operability disruption. The rest categories of events results in slightly higher average money lost in commercial sector.

The patterns of average money lost under different categories of events are consistent for all three sectors, indicating that the power outages caused by different categories of events have the same relative influence on the money lost for all three sectors.

### Hypothesis Testing

In [ ]:
outage_res = outage[['RES.MONEY.LOST']].rename({'RES.MONEY.LOST': 'MONEY.LOST'}, axis=1)
outage_res['SECTOR'] = ['residential']*outage_res.shape[0]
outage_com = outage[['COM.MONEY.LOST']].rename({'COM.MONEY.LOST': 'MONEY.LOST'}, axis=1)
outage_com['SECTOR'] = ['commercial']*outage_com.shape[0]
shuffled = pd.concat([outage_res, outage_com])

N = 1000
means = np.empty(N, dtype=float)
observed = shuffled.groupby('SECTOR').mean().diff().iloc[-1]['MONEY.LOST']
for i in range(N):
    shuffled['SECTOR'] = np.random.permutation(shuffled['SECTOR'])
    means[i] = shuffled.groupby('SECTOR').mean().diff().iloc[-1]['MONEY.LOST']
p_value = (means >= observed).mean()
print('p-value: ', p_value)
fig = px.histogram(pd.DataFrame(means), histnorm='probability', title='')
fig.add_vline(x=observed, line_color='red')
fig.show()
fig.write_html('plots/hypothesis_testing.html', include_plotlyjs='cdn')

Unfortunately, the pivot table cannot tell a clear difference between the overall money lost in the commercial sector and the residential sector. Given that the average money lost is higher in residential sector under more categories of events, we decide to use permutation testing to further investigate whether the money lost in residential sector is larger than the money lost in commercial sector. 

The null hypothesis is that, the average money lost in the residential sector is equal to the average money lost in the commercial sector during power outages . The alternative hypothesis is that, the average money lost in the residential sector is larger than the average money lost in the commercial sector during power outages. For the test statistic, we use the difference of the mean of two samples.

The p-value calculated using the simulation is 0.367. Under significance level of 0.05, we fail to reject null hypothesis. This result suggests that, although there are slight differences in the average money lost from residential sector and from commercial sectors under different categories of events that cause the power outage, we cannot say that the average money lost from two sectors are different.